In [3]:

!pip install snowflake-connector-python


In [4]:
!pip install "snowflake-connector-python[pandas]"

In [5]:
!pip install credentials

In [2]:
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import pandas as pd


In [7]:
import credentials

In [3]:
conn = snowflake.connector.connect(
    user = 'ashika',
    password = 'Cervello123#',
    account = 'bpwmwqd-bk67062',
    warehouse = 'compute_wh',
    database = 'RAW',
    schema = 'test',
)

In [4]:
cur = conn.cursor()

In [10]:
data = cur.execute('select * from users')

In [11]:
fdata = data.fetchall()

In [12]:
fdata

[(1, 'admin_user', 'admin@example.com', 'hashed_pw_1', 'admin'),
 (2, 'trainer_john', 'john_trainer@example.com', 'hashed_pw_2', 'trainer'),
 (3, 'student_anna', 'anna_student@example.com', 'hashed_pw_3', 'student'),
 (4, 'student_mark', 'mark_student@example.com', 'hashed_pw_4', 'student'),
 (5, 'trainer_emma', 'emma_trainer@example.com', 'hashed_pw_5', 'trainer')]

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.Builder().master('local[*]').appName("Pranav").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [28]:
columns = ['id','Name','email','role_id','role']

In [43]:
df = spark.createDataFrame(fdata, schema=columns)

In [44]:
df.show()

+---+------------+--------------------+-----------+-------+
| id|        Name|               email|    role_id|   role|
+---+------------+--------------------+-----------+-------+
|  1|  admin_user|   admin@example.com|hashed_pw_1|  admin|
|  2|trainer_john|john_trainer@exam...|hashed_pw_2|trainer|
|  3|student_anna|anna_student@exam...|hashed_pw_3|student|
|  4|student_mark|mark_student@exam...|hashed_pw_4|student|
|  5|trainer_emma|emma_trainer@exam...|hashed_pw_5|trainer|
+---+------------+--------------------+-----------+-------+



In [5]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'TEST' AND table_type = 'BASE TABLE'
""")

tables = cur.fetchall()
table_names = [table[0] for table in tables]

# Initialize an empty dictionary to store data from all tables
all_data = {}

# Fetch data from all tables
for table_name in table_names:
    cur.execute(f"SELECT * FROM {table_name} LIMIT 100")  # Limit the rows for simplicity
    data2 = cur.fetchall()
    df = pd.DataFrame(data2, columns=[col[0] for col in cur.description])
    all_data[table_name] = df


# Display the first few rows from all tables (for review)
for table, data2 in all_data.items():
    print(f"\nData from table: {table}")
    print(data2.head())


Data from table: CUSTOMERS
Empty DataFrame
Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
Index: []

Data from table: FEEDBACK
Empty DataFrame
Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
Index: []

Data from table: INVENTORY
Empty DataFrame
Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
Index: []

Data from table: ORDERS
Empty DataFrame
Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
Index: []

Data from table: PAYMENTS
Empty DataFrame
Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
Index: []

Data from table: PRODUCTS
Empty DataFrame
Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
Index: []

Data from table: RETURNS
Empty DataFrame
Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
Index: []

Data from ta

In [6]:
cur.description

[ResultMetadata(name='SALE_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=False),
 ResultMetadata(name='PRODUCT_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='CUSTOMER_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_DATE', type_code=6, display_size=None, internal_size=None, precision=0, scale=9, is_nullable=True),
 ResultMetadata(name='QUANTITY_SOLD', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_PRICE', type_code=0, display_size=None, internal_size=None, precision=10, scale=2, is_nullable=True),
 ResultMetadata(name='DISCOUNT', type_code=0, display_size=None, internal_size=None, precision=5, scale=2, is_nullable=True)]

In [7]:
all_data

{'CUSTOMERS': Empty DataFrame
 Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
 Index: [],
 'FEEDBACK': Empty DataFrame
 Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
 Index: [],
 'INVENTORY': Empty DataFrame
 Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
 Index: [],
 'ORDERS': Empty DataFrame
 Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
 Index: [],
 'PAYMENTS': Empty DataFrame
 Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
 Index: [],
 'PRODUCTS': Empty DataFrame
 Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
 Index: [],
 'RETURNS': Empty DataFrame
 Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
 Index: [],
 'SHIPPING': Empty DataFrame
 Columns: [SHIPPING_ID, ORDER_ID, SHIPPING_METHOD, SHIPPING_D

In [ ]:
cur.execute("""
    select get_ddl('SCHEMA','TEST');
""")

ff = cur.fetchall()

ff

[('create or replace schema TEST;\n\ncreate or replace TABLE CUSTOMERS (\n\tCUSTOMER_ID NUMBER(38,0) NOT NULL,\n\tFIRST_NAME VARCHAR(16777216),\n\tLAST_NAME VARCHAR(16777216),\n\tEMAIL VARCHAR(16777216),\n\tPHONE VARCHAR(16777216),\n\tADDRESS VARCHAR(16777216),\n\tJOIN_DATE TIMESTAMP_LTZ(9),\n\tprimary key (CUSTOMER_ID)\n);\ncreate or replace TABLE FEEDBACK (\n\tFEEDBACK_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tPRODUCT_ID NUMBER(38,0),\n\tRATING NUMBER(38,0),\n\tCOMMENTS VARCHAR(16777216),\n\tFEEDBACK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (FEEDBACK_ID)\n);\ncreate or replace TABLE INVENTORY (\n\tINVENTORY_ID NUMBER(38,0) NOT NULL,\n\tPRODUCT_ID NUMBER(38,0),\n\tWAREHOUSE_LOCATION VARCHAR(16777216),\n\tQUANTITY_ON_HAND NUMBER(38,0),\n\tLAST_RESTOCK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (INVENTORY_ID)\n);\ncreate or replace TABLE ORDERS (\n\tORDER_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tORDER_DATE TIMESTAMP_LTZ(9),\n\tSHIPPING_ADDRESS VARCHAR(16777216),\n\tTOTA

In [6]:

cur.execute("""
    SELECT 
    TABLE_NAME, 
    COLUMN_NAME, 
    DATA_TYPE, 
    IS_NULLABLE, 
    COLUMN_DEFAULT 
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_SCHEMA = 'TEST'
    order by table_name
;
""")

metadata = cur.fetchall()

metadata

[('CUSTOMERS', 'CUSTOMER_ID', 'NUMBER', 'NO', None),
 ('CUSTOMERS', 'FIRST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'LAST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'EMAIL', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'PHONE', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'ADDRESS', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'JOIN_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('FEEDBACK', 'USER_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_ID', 'NUMBER', 'NO', None),
 ('FEEDBACK', 'RATING', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'COMMENTS', 'TEXT', 'YES', None),
 ('INVENTORY', 'LAST_RESTOCK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('INVENTORY', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('INVENTORY', 'INVENTORY_ID', 'NUMBER', 'NO', None),
 ('INVENTORY', 'QUANTITY_ON_HAND', 'NUMBER', 'YES', None),
 ('INVENTORY', 'WAREHOUSE_LOCATION', 'TEXT', 'YES', None),
 ('ORDERS', 'PAYMENT_METHOD', 'TEXT', 'YES',

In [7]:
import json

In [27]:
tables

{'CUSTOMERS': [{'column_name': 'CUSTOMER_ID',
   'data_type': 'NUMBER',
   'is_nullable': 'NO'},
  {'column_name': 'FIRST_NAME', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'LAST_NAME', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'EMAIL', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'PHONE', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'ADDRESS', 'data_type': 'TEXT', 'is_nullable': 'YES'},
  {'column_name': 'JOIN_DATE',
   'data_type': 'TIMESTAMP_LTZ',
   'is_nullable': 'YES'}],
 'FEEDBACK': [{'column_name': 'FEEDBACK_DATE',
   'data_type': 'TIMESTAMP_LTZ',
   'is_nullable': 'YES'},
  {'column_name': 'USER_ID', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'PRODUCT_ID', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'FEEDBACK_ID', 'data_type': 'NUMBER', 'is_nullable': 'NO'},
  {'column_name': 'RATING', 'data_type': 'NUMBER', 'is_nullable': 'YES'},
  {'column_name': 'COMMENTS', 'data

In [75]:
tables = {}

# Loop through each column data tuple
for table_name, column_name, data_type, is_nullable, default_value in metadata:
    # Initialize a new table in the dictionary if it doesn't exist
    if table_name not in tables:
        tables[table_name] = {"table_name": table_name, "columns": []}
    
    # Add a new entry for the column in the table
    column_entry = {
        "name": column_name,
        "type": data_type,
        "constraints": []
    }
    
    # Add constraints based on metadata
    if not is_nullable:
        column_entry["constraints"].append("NOT NULL")
    if default_value:
        column_entry["constraints"].append(f"DEFAULT {default_value}")
    
    # Add example data for certain columns (optional, as per your format)
    if column_name == "FIRST_NAME":
        column_entry["example"] = "John"
    elif column_name == "LAST_NAME":
        column_entry["example"] = "Doe"
    elif column_name == "EMAIL":
        column_entry["example"] = "john.doe@example.com"
    elif column_name == "PHONE":
        column_entry["example"] = "123-456-7890"
    elif column_name == "ADDRESS":
        column_entry["example"] = "123 Elm St, Springfield, IL"
    elif column_name == "JOIN_DATE":
        column_entry["example"] = "2022-01-15T08:30:00Z"

    tables[table_name]["columns"].append(column_entry)

# Convert the tables dictionary to JSON format
json_data = json.dumps(tables, indent=2)

# Output the JSON data
print(json_data)


{
  "CUSTOMERS": {
    "table_name": "CUSTOMERS",
    "columns": [
      {
        "name": "CUSTOMER_ID",
        "type": "NUMBER",
        "constraints": []
      },
      {
        "name": "FIRST_NAME",
        "type": "TEXT",
        "constraints": [],
        "example": "John"
      },
      {
        "name": "LAST_NAME",
        "type": "TEXT",
        "constraints": [],
        "example": "Doe"
      },
      {
        "name": "EMAIL",
        "type": "TEXT",
        "constraints": [],
        "example": "john.doe@example.com"
      },
      {
        "name": "PHONE",
        "type": "TEXT",
        "constraints": [],
        "example": "123-456-7890"
      },
      {
        "name": "ADDRESS",
        "type": "TEXT",
        "constraints": [],
        "example": "123 Elm St, Springfield, IL"
      },
      {
        "name": "JOIN_DATE",
        "type": "TIMESTAMP_LTZ",
        "constraints": [],
        "example": "2022-01-15T08:30:00Z"
      }
    ]
  },
  "FEEDBACK": {
    "ta

In [9]:
from langchain_openai import AzureChatOpenAI
import os

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_4o_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
)

In [2]:
from langchain_core.prompts import PromptTemplate

template = (
    '''You are a data generator tasked with creating synthetic data. 
    Based on the following JSON metadata describing table structure and data types, generate sample data rows for each column. 
    Ensure the data adheres to the specified types, constraints, and formats. 
    Provide 10 rows of sample data in JSON array format, and ensure it is realistic and coherent.
    {json_data}
    '''
)

prompt = PromptTemplate.from_template(template)

In [73]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
You are a data generator tasked with creating synthetic data. Based on the following JSON metadata describing table structure and data types, generate sample data rows for each column. 
- Adhere to the specified types, constraints, and formats.
- Provide 10 rows of sample data in JSON array format.
- Ensure the data is realistic and coherent.

Metadata:
{metadata}

Expected Output:
Provide 10 rows of JSON data for each table. use same format as metadata.
Provide the output in pure json format which I can parse as a json data to various platforms.
Generate json serializable data.

This is required format in which we require generated data.
please remove any additional content.
Provide only json data which is in curly braces.

"""

# Create the prompt
prompt = PromptTemplate(input_variables=["metadata"], template=prompt_template)
formatted_prompt = prompt.format(metadata=json_data)

# print(formatted_prompt)
response = model(formatted_prompt)

synthetic_data = response.content
# Display the result
print("Generated Synthetic Data:\n", synthetic_data)

# try:
#     data = json.loads(json_data)
# except json.JSONDecodeError:
#     print("Error: Failed to parse generated JSON data.")
#     data = []

# # Save to CSV if data is valid
# if data:
#     output_file = "synthetic_data.csv"
#     column_names = [col["name"] for col in json_data["columns"]]

#     with open(output_file, mode="w", newline="", encoding="utf-8") as file:
#         writer = csv.DictWriter(file, fieldnames=column_names)
#         writer.writeheader()
#         writer.writerows(data)

#     print(f"Data successfully saved to {output_file}")
# else:
#     print("No valid data generated.")

Generated Synthetic Data:
 ```json
{
  "CUSTOMERS": [
    {
      "CUSTOMER_ID": 1,
      "FIRST_NAME": "John",
      "LAST_NAME": "Doe",
      "EMAIL": "john.doe1@example.com",
      "PHONE": "123-456-7890",
      "ADDRESS": "123 Elm St, Springfield, IL",
      "JOIN_DATE": "2022-01-15T08:30:00Z"
    },
    {
      "CUSTOMER_ID": 2,
      "FIRST_NAME": "Jane",
      "LAST_NAME": "Smith",
      "EMAIL": "jane.smith@example.com",
      "PHONE": "234-567-8901",
      "ADDRESS": "456 Oak St, Springfield, IL",
      "JOIN_DATE": "2022-02-20T10:00:00Z"
    },
    {
      "CUSTOMER_ID": 3,
      "FIRST_NAME": "Michael",
      "LAST_NAME": "Johnson",
      "EMAIL": "michael.johnson@example.com",
      "PHONE": "345-678-9012",
      "ADDRESS": "789 Pine St, Springfield, IL",
      "JOIN_DATE": "2022-03-05T12:15:00Z"
    },
    {
      "CUSTOMER_ID": 4,
      "FIRST_NAME": "Emily",
      "LAST_NAME": "Davis",
      "EMAIL": "emily.davis@example.com",
      "PHONE": "456-789-0123",
      "ADDRES

In [38]:
import csv

In [104]:
try:
    data1 = json.loads(synthetic_data)
except json.JSONDecodeError:
    print("Error: Failed to parse generated JSON data.")
    data1 = []

data1

Error: Failed to parse generated JSON data.


[]

In [60]:
type(data)

dict

In [61]:
import json

if isinstance(data, str):  # Check if 'data' is a string
    data = json.loads(data)  # Convert JSON string to Python object

In [100]:
# Save to CSV if data is valid
if data1:
    output_file = "synthetic_data.csv"
    for table in data1.values():
        column_names = [column["name"] for column in table["columns"]]

        with open(output_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=column_names)
            writer.writeheader()
            writer.writerows(data)

            print(f"Data successfully saved to {output_file}")
else:
            print("No valid data generated.")

AttributeError: 'str' object has no attribute 'keys'

In [96]:
data = json.dumps(synthetic_data)

In [97]:
print(json.loads(data))

```json
{
  "CUSTOMERS": [
    {
      "CUSTOMER_ID": 1,
      "FIRST_NAME": "John",
      "LAST_NAME": "Doe",
      "EMAIL": "john.doe1@example.com",
      "PHONE": "123-456-7890",
      "ADDRESS": "123 Elm St, Springfield, IL",
      "JOIN_DATE": "2022-01-15T08:30:00Z"
    },
    {
      "CUSTOMER_ID": 2,
      "FIRST_NAME": "Jane",
      "LAST_NAME": "Smith",
      "EMAIL": "jane.smith@example.com",
      "PHONE": "234-567-8901",
      "ADDRESS": "456 Oak St, Springfield, IL",
      "JOIN_DATE": "2022-02-20T10:00:00Z"
    },
    {
      "CUSTOMER_ID": 3,
      "FIRST_NAME": "Michael",
      "LAST_NAME": "Johnson",
      "EMAIL": "michael.johnson@example.com",
      "PHONE": "345-678-9012",
      "ADDRESS": "789 Pine St, Springfield, IL",
      "JOIN_DATE": "2022-03-05T12:15:00Z"
    },
    {
      "CUSTOMER_ID": 4,
      "FIRST_NAME": "Emily",
      "LAST_NAME": "Davis",
      "EMAIL": "emily.davis@example.com",
      "PHONE": "456-789-0123",
      "ADDRESS": "101 Maple St, Springfi

In [112]:
import csv

# Data in JSON format
data = {
  "CUSTOMERS": [
    {
      "CUSTOMER_ID": 1,
      "FIRST_NAME": "John",
      "LAST_NAME": "Doe",
      "EMAIL": "john.doe1@example.com",
      "PHONE": "123-456-7890",
      "ADDRESS": "123 Elm St, Springfield, IL",
      "JOIN_DATE": "2022-01-15T08:30:00Z"
    },
    {
      "CUSTOMER_ID": 2,
      "FIRST_NAME": "Jane",
      "LAST_NAME": "Smith",
      "EMAIL": "jane.smith@example.com",
      "PHONE": "234-567-8901",
      "ADDRESS": "456 Oak St, Springfield, IL",
      "JOIN_DATE": "2022-02-20T10:00:00Z"
    },
    {
      "CUSTOMER_ID": 3,
      "FIRST_NAME": "Michael",
      "LAST_NAME": "Johnson",
      "EMAIL": "michael.johnson@example.com",
      "PHONE": "345-678-9012",
      "ADDRESS": "789 Pine St, Springfield, IL",
      "JOIN_DATE": "2022-03-05T12:15:00Z"
    },
    {
      "CUSTOMER_ID": 4,
      "FIRST_NAME": "Emily",
      "LAST_NAME": "Davis",
      "EMAIL": "emily.davis@example.com",
      "PHONE": "456-789-0123",
      "ADDRESS": "101 Maple St, Springfield, IL",
      "JOIN_DATE": "2022-04-10T14:45:00Z"
    },
    {
      "CUSTOMER_ID": 5,
      "FIRST_NAME": "David",
      "LAST_NAME": "Brown",
      "EMAIL": "david.brown@example.com",
      "PHONE": "567-890-1234",
      "ADDRESS": "202 Birch St, Springfield, IL",
      "JOIN_DATE": "2022-05-15T16:30:00Z"
    },
    {
      "CUSTOMER_ID": 6,
      "FIRST_NAME": "Sophia",
      "LAST_NAME": "Wilson",
      "EMAIL": "sophia.wilson@example.com",
      "PHONE": "678-901-2345",
      "ADDRESS": "303 Cedar St, Springfield, IL",
      "JOIN_DATE": "2022-06-20T18:00:00Z"
    },
    {
      "CUSTOMER_ID": 7,
      "FIRST_NAME": "James",
      "LAST_NAME": "Martinez",
      "EMAIL": "james.martinez@example.com",
      "PHONE": "789-012-3456",
      "ADDRESS": "404 Spruce St, Springfield, IL",
      "JOIN_DATE": "2022-07-25T20:15:00Z"
    },
    {
      "CUSTOMER_ID": 8,
      "FIRST_NAME": "Olivia",
      "LAST_NAME": "Anderson",
      "EMAIL": "olivia.anderson@example.com",
      "PHONE": "890-123-4567",
      "ADDRESS": "505 Walnut St, Springfield, IL",
      "JOIN_DATE": "2022-08-30T22:30:00Z"
    },
    {
      "CUSTOMER_ID": 9,
      "FIRST_NAME": "Lucas",
      "LAST_NAME": "Thomas",
      "EMAIL": "lucas.thomas@example.com",
      "PHONE": "901-234-5678",
      "ADDRESS": "606 Willow St, Springfield, IL",
      "JOIN_DATE": "2022-09-15T07:00:00Z"
    },
    {
      "CUSTOMER_ID": 10,
      "FIRST_NAME": "Mia",
      "LAST_NAME": "Taylor",
      "EMAIL": "mia.taylor@example.com",
      "PHONE": "012-345-6789",
      "ADDRESS": "707 Chestnut St, Springfield, IL",
      "JOIN_DATE": "2022-10-20T09:30:00Z"
    }
  ],
  "FEEDBACK": [
    {
      "FEEDBACK_DATE": "2023-01-01T10:00:00Z",
      "USER_ID": 1,
      "PRODUCT_ID": 101,
      "FEEDBACK_ID": 1,
      "RATING": 5,
      "COMMENTS": "Excellent product!"
    },
    {
      "FEEDBACK_DATE": "2023-01-02T11:30:00Z",
      "USER_ID": 2,
      "PRODUCT_ID": 102,
      "FEEDBACK_ID": 2,
      "RATING": 4,
      "COMMENTS": "Very good, but could be improved."
    },
    {
      "FEEDBACK_DATE": "2023-01-03T14:00:00Z",
      "USER_ID": 3,
      "PRODUCT_ID": 103,
      "FEEDBACK_ID": 3,
      "RATING": 3,
      "COMMENTS": "Average product."
    },
    {
      "FEEDBACK_DATE": "2023-01-04T15:45:00Z",
      "USER_ID": 4,
      "PRODUCT_ID": 104,
      "FEEDBACK_ID": 4,
      "RATING": 2,
      "COMMENTS": "Not satisfied with the quality."
    },
    {
      "FEEDBACK_DATE": "2023-01-05T17:30:00Z",
      "USER_ID": 5,
      "PRODUCT_ID": 105,
      "FEEDBACK_ID": 5,
      "RATING": 1,
      "COMMENTS": "Poor product, would not recommend."
    },
    {
      "FEEDBACK_DATE": "2023-01-06T09:00:00Z",
      "USER_ID": 6,
      "PRODUCT_ID": 106,
      "FEEDBACK_ID": 6,
      "RATING": 5,
      "COMMENTS": "Fantastic! Will buy again."
    },
    {
      "FEEDBACK_DATE": "2023-01-07T10:30:00Z",
      "USER_ID": 7,
      "PRODUCT_ID": 107,
      "FEEDBACK_ID": 7,
      "RATING": 4,
      "COMMENTS": "Good product, satisfied with the purchase."
    },
    {
      "FEEDBACK_DATE": "2023-01-08T13:15:00Z",
      "USER_ID": 8,
      "PRODUCT_ID": 108,
      "FEEDBACK_ID": 8,
      "RATING": 3,
      "COMMENTS": "It's okay, does the job."
    },
    {
      "FEEDBACK_DATE": "2023-01-09T16:45:00Z",
      "USER_ID": 9,
      "PRODUCT_ID": 109,
      "FEEDBACK_ID": 9,
      "RATING": 2,
      "COMMENTS": "Could be better, not what I expected."
    },
    {
      "FEEDBACK_DATE": "2023-01-10T18:00:00Z",
      "USER_ID": 10,
      "PRODUCT_ID": 110,
      "FEEDBACK_ID": 10,
      "RATING": 1,
      "COMMENTS": "Very disappointed with this product."
    }
  ],
  "INVENTORY": [
    {
      "LAST_RESTOCK_DATE": "2023-01-01T08:00:00Z",
      "PRODUCT_ID": 101,
      "INVENTORY_ID": 1,
      "QUANTITY_ON_HAND": 100,
      "WAREHOUSE_LOCATION": "A1"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-02T09:00:00Z",
      "PRODUCT_ID": 102,
      "INVENTORY_ID": 2,
      "QUANTITY_ON_HAND": 150,
      "WAREHOUSE_LOCATION": "B2"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-03T10:00:00Z",
      "PRODUCT_ID": 103,
      "INVENTORY_ID": 3,
      "QUANTITY_ON_HAND": 200,
      "WAREHOUSE_LOCATION": "C3"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-04T11:00:00Z",
      "PRODUCT_ID": 104,
      "INVENTORY_ID": 4,
      "QUANTITY_ON_HAND": 250,
      "WAREHOUSE_LOCATION": "D4"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-05T12:00:00Z",
      "PRODUCT_ID": 105,
      "INVENTORY_ID": 5,
      "QUANTITY_ON_HAND": 300,
      "WAREHOUSE_LOCATION": "E5"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-06T13:00:00Z",
      "PRODUCT_ID": 106,
      "INVENTORY_ID": 6,
      "QUANTITY_ON_HAND": 350,
      "WAREHOUSE_LOCATION": "F6"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-07T14:00:00Z",
      "PRODUCT_ID": 107,
      "INVENTORY_ID": 7,
      "QUANTITY_ON_HAND": 400,
      "WAREHOUSE_LOCATION": "G7"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-08T15:00:00Z",
      "PRODUCT_ID": 108,
      "INVENTORY_ID": 8,
      "QUANTITY_ON_HAND": 450,
      "WAREHOUSE_LOCATION": "H8"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-09T16:00:00Z",
      "PRODUCT_ID": 109,
      "INVENTORY_ID": 9,
      "QUANTITY_ON_HAND": 500,
      "WAREHOUSE_LOCATION": "I9"
    },
    {
      "LAST_RESTOCK_DATE": "2023-01-10T17:00:00Z",
      "PRODUCT_ID": 110,
      "INVENTORY_ID": 10,
      "QUANTITY_ON_HAND": 550,
      "WAREHOUSE_LOCATION": "J10"
    }
  ],
  "ORDERS": [
    {
      "PAYMENT_METHOD": "Credit Card",
      "ORDER_STATUS": "Shipped",
      "TOTAL_AMOUNT": 150.75,
      "ORDER_ID": 1,
      "USER_ID": 1,
      "ORDER_DATE": "2023-01-01T09:30:00Z",
      "SHIPPING_ADDRESS": "123 Elm St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "PayPal",
      "ORDER_STATUS": "Processing",
      "TOTAL_AMOUNT": 250.50,
      "ORDER_ID": 2,
      "USER_ID": 2,
      "ORDER_DATE": "2023-01-02T10:30:00Z",
      "SHIPPING_ADDRESS": "456 Oak St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Debit Card",
      "ORDER_STATUS": "Delivered",
      "TOTAL_AMOUNT": 350.25,
      "ORDER_ID": 3,
      "USER_ID": 3,
      "ORDER_DATE": "2023-01-03T11:30:00Z",
      "SHIPPING_ADDRESS": "789 Pine St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Credit Card",
      "ORDER_STATUS": "Canceled",
      "TOTAL_AMOUNT": 450.00,
      "ORDER_ID": 4,
      "USER_ID": 4,
      "ORDER_DATE": "2023-01-04T12:30:00Z",
      "SHIPPING_ADDRESS": "101 Maple St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "PayPal",
      "ORDER_STATUS": "Shipped",
      "TOTAL_AMOUNT": 550.75,
      "ORDER_ID": 5,
      "USER_ID": 5,
      "ORDER_DATE": "2023-01-05T13:30:00Z",
      "SHIPPING_ADDRESS": "202 Birch St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Debit Card",
      "ORDER_STATUS": "Processing",
      "TOTAL_AMOUNT": 650.50,
      "ORDER_ID": 6,
      "USER_ID": 6,
      "ORDER_DATE": "2023-01-06T14:30:00Z",
      "SHIPPING_ADDRESS": "303 Cedar St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Credit Card",
      "ORDER_STATUS": "Delivered",
      "TOTAL_AMOUNT": 750.25,
      "ORDER_ID": 7,
      "USER_ID": 7,
      "ORDER_DATE": "2023-01-07T15:30:00Z",
      "SHIPPING_ADDRESS": "404 Spruce St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "PayPal",
      "ORDER_STATUS": "Canceled",
      "TOTAL_AMOUNT": 850.00,
      "ORDER_ID": 8,
      "USER_ID": 8,
      "ORDER_DATE": "2023-01-08T16:30:00Z",
      "SHIPPING_ADDRESS": "505 Walnut St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Debit Card",
      "ORDER_STATUS": "Shipped",
      "TOTAL_AMOUNT": 950.75,
      "ORDER_ID": 9,
      "USER_ID": 9,
      "ORDER_DATE": "2023-01-09T17:30:00Z",
      "SHIPPING_ADDRESS": "606 Willow St, Springfield, IL"
    },
    {
      "PAYMENT_METHOD": "Credit Card",
      "ORDER_STATUS": "Processing",
      "TOTAL_AMOUNT": 1050.50,
      "ORDER_ID": 10,
      "USER_ID": 10,
      "ORDER_DATE": "2023-01-10T18:30:00Z",
      "SHIPPING_ADDRESS": "707 Chestnut St, Springfield, IL"
    }
  ],
  "PAYMENTS": [
    {
      "PAYMENT_DATE": "2023-01-01T09:45:00Z",
      "PAYMENT_METHOD": "Credit Card",
      "PAYMENT_ID": 1,
      "PAYMENT_STATUS": "Completed",
      "PAYMENT_AMOUNT": 150.75,
      "ORDER_ID": 1
    },
    {
      "PAYMENT_DATE": "2023-01-02T10:45:00Z",
      "PAYMENT_METHOD": "PayPal",
      "PAYMENT_ID": 2,
      "PAYMENT_STATUS": "Pending",
      "PAYMENT_AMOUNT": 250.50,
      "ORDER_ID": 2
    },
    {
      "PAYMENT_DATE": "2023-01-03T11:45:00Z",
      "PAYMENT_METHOD": "Debit Card",
      "PAYMENT_ID": 3,
      "PAYMENT_STATUS": "Completed",
      "PAYMENT_AMOUNT": 350.25,
      "ORDER_ID": 3
    },
    {
      "PAYMENT_DATE": "2023-01-04T12:45:00Z",
      "PAYMENT_METHOD": "Credit Card",
      "PAYMENT_ID": 4,
      "PAYMENT_STATUS": "Canceled",
      "PAYMENT_AMOUNT": 450.00,
      "ORDER_ID": 4
    },
    {
      "PAYMENT_DATE": "2023-01-05T13:45:00Z",
      "PAYMENT_METHOD": "PayPal",
      "PAYMENT_ID": 5,
      "PAYMENT_STATUS": "Completed",
      "PAYMENT_AMOUNT": 550.75,
      "ORDER_ID": 5
    },
    {
      "PAYMENT_DATE": "2023-01-06T14:45:00Z",
      "PAYMENT_METHOD": "Debit Card",
      "PAYMENT_ID": 6,
      "PAYMENT_STATUS": "Pending",
      "PAYMENT_AMOUNT": 650.50,
      "ORDER_ID": 6
    },
    {
      "PAYMENT_DATE": "2023-01-07T15:45:00Z",
      "PAYMENT_METHOD": "Credit Card",
      "PAYMENT_ID": 7,
      "PAYMENT_STATUS": "Completed",
      "PAYMENT_AMOUNT": 750.25,
      "ORDER_ID": 7
    },
    {
      "PAYMENT_DATE": "2023-01-08T16:45:00Z",
      "PAYMENT_METHOD": "PayPal",
      "PAYMENT_ID": 8,
      "PAYMENT_STATUS": "Canceled",
      "PAYMENT_AMOUNT": 850.00,
      "ORDER_ID": 8
    },
    {
      "PAYMENT_DATE": "2023-01-09T17:45:00Z",
      "PAYMENT_METHOD": "Debit Card",
      "PAYMENT_ID": 9,
      "PAYMENT_STATUS": "Completed",
      "PAYMENT_AMOUNT": 950.75,
      "ORDER_ID": 9
    },
    {
      "PAYMENT_DATE": "2023-01-10T18:45:00Z",
      "PAYMENT_METHOD": "Credit Card",
      "PAYMENT_ID": 10,
      "PAYMENT_STATUS": "Pending",
      "PAYMENT_AMOUNT": 1050.50,
      "ORDER_ID": 10
    }
  ],
  "PRODUCTS": [
    {
      "PRODUCT_NAME": "Widget A",
      "PRODUCT_ID": 101,
      "PRICE": 19.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 100,
      "RELEASE_DATE": "2021-01-01",
      "CATEGORY": "Gadgets"
    },
    {
      "PRODUCT_NAME": "Widget B",
      "PRODUCT_ID": 102,
      "PRICE": 29.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 150,
      "RELEASE_DATE": "2021-02-15",
      "CATEGORY": "Gadgets"
    },
    {
      "PRODUCT_NAME": "Widget C",
      "PRODUCT_ID": 103,
      "PRICE": 39.99,
      "IS_ACTIVE": "false",
      "STOCK_COUNT": 200,
      "RELEASE_DATE": "2021-03-01",
      "CATEGORY": "Gadgets"
    },
    {
      "PRODUCT_NAME": "Gadget X",
      "PRODUCT_ID": 104,
      "PRICE": 49.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 250,
      "RELEASE_DATE": "2021-04-10",
      "CATEGORY": "Accessories"
    },
    {
      "PRODUCT_NAME": "Gadget Y",
      "PRODUCT_ID": 105,
      "PRICE": 59.99,
      "IS_ACTIVE": "false",
      "STOCK_COUNT": 300,
      "RELEASE_DATE": "2021-05-20",
      "CATEGORY": "Accessories"
    },
    {
      "PRODUCT_NAME": "Gadget Z",
      "PRODUCT_ID": 106,
      "PRICE": 69.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 350,
      "RELEASE_DATE": "2021-06-30",
      "CATEGORY": "Accessories"
    },
    {
      "PRODUCT_NAME": "Device 1",
      "PRODUCT_ID": 107,
      "PRICE": 79.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 400,
      "RELEASE_DATE": "2021-07-15",
      "CATEGORY": "Devices"
    },
    {
      "PRODUCT_NAME": "Device 2",
      "PRODUCT_ID": 108,
      "PRICE": 89.99,
      "IS_ACTIVE": "false",
      "STOCK_COUNT": 450,
      "RELEASE_DATE": "2021-08-25",
      "CATEGORY": "Devices"
    },
    {
      "PRODUCT_NAME": "Device 3",
      "PRODUCT_ID": 109,
      "PRICE": 99.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 500,
      "RELEASE_DATE": "2021-09-05",
      "CATEGORY": "Devices"
    },
    {
      "PRODUCT_NAME": "Appliance A",
      "PRODUCT_ID": 110,
      "PRICE": 109.99,
      "IS_ACTIVE": "true",
      "STOCK_COUNT": 550,
      "RELEASE_DATE": "2021-10-15",
      "CATEGORY": "Appliances"
    }
  ],
  "RETURNS": [
    {
      "REASON": "Defective item",
      "ORDER_ID": 1,
      "RETURN_ID": 1,
      "PRODUCT_ID": 101,
      "REFUND_AMOUNT": 19.99,
      "RETURN_DATE": "2023-01-05T10:00:00Z"
    },
    {
      "REASON": "Wrong item sent",
      "ORDER_ID": 2,
      "RETURN_ID": 2,
      "PRODUCT_ID": 102,
      "REFUND_AMOUNT": 29.99,
      "RETURN_DATE": "2023-01-06T11:00:00Z"
    },
    {
      "REASON": "Item not as described",
      "ORDER_ID": 3,
      "RETURN_ID": 3,
      "PRODUCT_ID": 103,
      "REFUND_AMOUNT": 39.99,
      "RETURN_DATE": "2023-01-07T12:00:00Z"
    },
    {
      "REASON": "Overcharged",
      "ORDER_ID": 4,
      "RETURN_ID": 4,
      "PRODUCT_ID": 104,
      "REFUND_AMOUNT": 49.99,
      "RETURN_DATE": "2023-01-08T13:00:00Z"
    },
    {
      "REASON": "Product damaged",
      "ORDER_ID": 5,
      "RETURN_ID": 5,
      "PRODUCT_ID": 105,
      "REFUND_AMOUNT": 59.99,
      "RETURN_DATE": "2023-01-09T14:00:00Z"
    },
    {
      "REASON": "Not satisfied",
      "ORDER_ID": 6,
      "RETURN_ID": 6,
      "PRODUCT_ID": 106,
      "REFUND_AMOUNT": 69.99,
      "RETURN_DATE": "2023-01-10T15:00:00Z"
    },
    {
      "REASON": "Better price elsewhere",
      "ORDER_ID": 7,
      "RETURN_ID": 7,
      "PRODUCT_ID": 107,
      "REFUND_AMOUNT": 79.99,
      "RETURN_DATE": "2023-01-11T16:00:00Z"
    },
    {
      "REASON": "Late delivery",
      "ORDER_ID": 8,
      "RETURN_ID": 8,
      "PRODUCT_ID": 108,
      "REFUND_AMOUNT": 89.99,
      "RETURN_DATE": "2023-01-12T17:00:00Z"
    },
    {
      "REASON": "Changed mind",
      "ORDER_ID": 9,
      "RETURN_ID": 9,
      "PRODUCT_ID": 109,
      "REFUND_AMOUNT": 99.99,
      "RETURN_DATE": "2023-01-13T18:00:00Z"
    },
    {
      "REASON": "Found a defect",
      "ORDER_ID": 10,
      "RETURN_ID": 10,
      "PRODUCT_ID": 110,
      "REFUND_AMOUNT": 109.99,
      "RETURN_DATE": "2023-01-14T19:00:00Z"
    }
  ],
  "SALES": [
    {
      "PRODUCT_ID": 101,
      "CUSTOMER_ID": 1,
      "SALE_PRICE": 18.99,
      "SALE_DATE": "2023-01-01T09:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 1,
      "QUANTITY_SOLD": 1
    },
    {
      "PRODUCT_ID": 102,
      "CUSTOMER_ID": 2,
      "SALE_PRICE": 28.99,
      "SALE_DATE": "2023-01-02T10:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 2,
      "QUANTITY_SOLD": 2
    },
    {
      "PRODUCT_ID": 103,
      "CUSTOMER_ID": 3,
      "SALE_PRICE": 38.99,
      "SALE_DATE": "2023-01-03T11:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 3,
      "QUANTITY_SOLD": 3
    },
    {
      "PRODUCT_ID": 104,
      "CUSTOMER_ID": 4,
      "SALE_PRICE": 48.99,
      "SALE_DATE": "2023-01-04T12:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 4,
      "QUANTITY_SOLD": 4
    },
    {
      "PRODUCT_ID": 105,
      "CUSTOMER_ID": 5,
      "SALE_PRICE": 58.99,
      "SALE_DATE": "2023-01-05T13:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 5,
      "QUANTITY_SOLD": 5
    },
    {
      "PRODUCT_ID": 106,
      "CUSTOMER_ID": 6,
      "SALE_PRICE": 68.99,
      "SALE_DATE": "2023-01-06T14:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 6,
      "QUANTITY_SOLD": 6
    },
    {
      "PRODUCT_ID": 107,
      "CUSTOMER_ID": 7,
      "SALE_PRICE": 78.99,
      "SALE_DATE": "2023-01-07T15:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 7,
      "QUANTITY_SOLD": 7
    },
    {
      "PRODUCT_ID": 108,
      "CUSTOMER_ID": 8,
      "SALE_PRICE": 88.99,
      "SALE_DATE": "2023-01-08T16:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 8,
      "QUANTITY_SOLD": 8
    },
    {
      "PRODUCT_ID": 109,
      "CUSTOMER_ID": 9,
      "SALE_PRICE": 98.99,
      "SALE_DATE": "2023-01-09T17:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 9,
      "QUANTITY_SOLD": 9
    },
    {
      "PRODUCT_ID": 110,
      "CUSTOMER_ID": 10,
      "SALE_PRICE": 108.99,
      "SALE_DATE": "2023-01-10T18:00:00Z",
      "DISCOUNT": 1.00,
      "SALE_ID": 10,
      "QUANTITY_SOLD": 10
    }
  ],
  "SHIPPING": [
    {
      "TRACKING_NUMBER": "1Z9999W99999999999",
      "SHIPPING_DATE": "2023-01-01T10:00:00Z",
      "SHIPPING_STATUS": "In Transit",
      "SHIPPING_METHOD": "UPS",
      "DELIVERY_DATE": "2023-01-03T10:00:00Z",
      "ORDER_ID": 1,
      "SHIPPING_ID": 1
    },
    {
      "TRACKING_NUMBER": "1Z8888W88888888888",
      "SHIPPING_DATE": "2023-01-02T10:00:00Z",
      "SHIPPING_STATUS": "Delivered",
      "SHIPPING_METHOD": "FedEx",
      "DELIVERY_DATE": "2023-01-04T10:00:00Z",
      "ORDER_ID": 2,
      "SHIPPING_ID": 2
    },
    {
      "TRACKING_NUMBER": "1Z7777W77777777777",
      "SHIPPING_DATE": "2023-01-03T10:00:00Z",
      "SHIPPING_STATUS": "Pending",
      "SHIPPING_METHOD": "USPS",
      "DELIVERY_DATE": "2023-01-05T10:00:00Z",
      "ORDER_ID": 3,
      "SHIPPING_ID": 3
    },
    {
      "TRACKING_NUMBER": "1Z6666W66666666666",
      "SHIPPING_DATE": "2023-01-04T10:00:00Z",
      "SHIPPING_STATUS": "In Transit",
      "SHIPPING_METHOD": "UPS",
      "DELIVERY_DATE": "2023-01-06T10:00:00Z",
      "ORDER_ID": 4,
      "SHIPPING_ID": 4
    },
    {
      "TRACKING_NUMBER": "1Z5555W55555555555",
      "SHIPPING_DATE": "2023-01-05T10:00:00Z",
      "SHIPPING_STATUS": "Delivered",
      "SHIPPING_METHOD": "FedEx",
      "DELIVERY_DATE": "2023-01-07T10:00:00Z",
      "ORDER_ID": 5,
      "SHIPPING_ID": 5
    },
    {
      "TRACKING_NUMBER": "1Z4444W44444444444",
      "SHIPPING_DATE": "2023-01-06T10:00:00Z",
      "SHIPPING_STATUS": "Pending",
      "SHIPPING_METHOD": "USPS",
      "DELIVERY_DATE": "2023-01-08T10:00:00Z",
      "ORDER_ID": 6,
      "SHIPPING_ID": 6
    },
    {
      "TRACKING_NUMBER": "1Z3333W33333333333",
      "SHIPPING_DATE": "2023-01-07T10:00:00Z",
      "SHIPPING_STATUS": "In Transit",
      "SHIPPING_METHOD": "UPS",
      "DELIVERY_DATE": "2023-01-09T10:00:00Z",
      "ORDER_ID": 7,
      "SHIPPING_ID": 7
    },
    {
      "TRACKING_NUMBER": "1Z2222W22222222222",
      "SHIPPING_DATE": "2023-01-08T10:00:00Z",
      "SHIPPING_STATUS": "Delivered",
      "SHIPPING_METHOD": "FedEx",
      "DELIVERY_DATE": "2023-01-10T10:00:00Z",
      "ORDER_ID": 8,
      "SHIPPING_ID": 8
    },
    {
      "TRACKING_NUMBER": "1Z1111W11111111111",
      "SHIPPING_DATE": "2023-01-09T10:00:00Z",
      "SHIPPING_STATUS": "Pending",
      "SHIPPING_METHOD": "USPS",
      "DELIVERY_DATE": "2023-01-11T10:00:00Z",
      "ORDER_ID": 9,
      "SHIPPING_ID": 9
    },
    {
      "TRACKING_NUMBER": "1Z0000W00000000000",
      "SHIPPING_DATE": "2023-01-10T10:00:00Z",
      "SHIPPING_STATUS": "In Transit",
      "SHIPPING_METHOD": "UPS",
      "DELIVERY_DATE": "2023-01-12T10:00:00Z",
      "ORDER_ID": 10,
      "SHIPPING_ID": 10
    }
  ],
  "USERS": [
    {
      "ROLE": "Admin",
      "USER_ID": 1,
      "USERNAME": "admin1",
      "PASSWORD": "pass1234",
      "EMAIL": "admin1@example.com"
    },
    {
      "ROLE": "Customer",
      "USER_ID": 2,
      "USERNAME": "customer1",
      "PASSWORD": "custpass1",
      "EMAIL": "customer1@example.com"
    },
    {
      "ROLE": "Customer",
      "USER_ID": 3,
      "USERNAME": "customer2",
      "PASSWORD": "custpass2",
      "EMAIL": "customer2@example.com"
    },
    {
      "ROLE": "Vendor",
      "USER_ID": 4,
      "USERNAME": "vendor1",
      "PASSWORD": "vendpass1",
      "EMAIL": "vendor1@example.com"
    },
    {
      "ROLE": "Vendor",
      "USER_ID": 5,
      "USERNAME": "vendor2",
      "PASSWORD": "vendpass2",
      "EMAIL": "vendor2@example.com"
    },
    {
      "ROLE": "Admin",
      "USER_ID": 6,
      "USERNAME": "admin2",
      "PASSWORD": "pass5678",
      "EMAIL": "admin2@example.com"
    },
    {
      "ROLE": "Customer",
      "USER_ID": 7,
      "USERNAME": "customer3",
      "PASSWORD": "custpass3",
      "EMAIL": "customer3@example.com"
    },
    {
      "ROLE": "Customer",
      "USER_ID": 8,
      "USERNAME": "customer4",
      "PASSWORD": "custpass4",
      "EMAIL": "customer4@example.com"
    },
    {
      "ROLE": "Vendor",
      "USER_ID": 9,
      "USERNAME": "vendor3",
      "PASSWORD": "vendpass3",
      "EMAIL": "vendor3@example.com"
    },
    {
      "ROLE": "Vendor",
      "USER_ID": 10,
      "USERNAME": "vendor4",
      "PASSWORD": "vendpass4",
      "EMAIL": "vendor4@example.com"
    }
  ]
}

output_dir = "csv_output"
os.makedirs(output_dir, exist_ok=True)

# Process each table
for table_name, rows in data.items():
    if rows:  # Check if the table has data
        # Define output CSV file path
        output_csv_file = os.path.join(output_dir, f"{table_name}.csv")
        
        # Get column names from the first row
        column_names = rows[0].keys()

        # Write data to CSV file
        with open(output_csv_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=column_names)
            
            # Write header and rows
            writer.writeheader()
            writer.writerows(rows)

        print(f"Table '{table_name}' saved to {output_csv_file}")
    else:
        print(f"Table '{table_name}' is empty. No file created.")

Table 'CUSTOMERS' saved to csv_output\CUSTOMERS.csv
Table 'FEEDBACK' saved to csv_output\FEEDBACK.csv
Table 'INVENTORY' saved to csv_output\INVENTORY.csv
Table 'ORDERS' saved to csv_output\ORDERS.csv
Table 'PAYMENTS' saved to csv_output\PAYMENTS.csv
Table 'PRODUCTS' saved to csv_output\PRODUCTS.csv
Table 'RETURNS' saved to csv_output\RETURNS.csv
Table 'SALES' saved to csv_output\SALES.csv
Table 'SHIPPING' saved to csv_output\SHIPPING.csv
Table 'USERS' saved to csv_output\USERS.csv


In [111]:
for table_name, rows in data.items():
    print(table_name)

CUSTOMERS
